In [1]:
import skfem
import skfem.helpers

In [2]:
mesh = skfem.Mesh.load("cube.msh")

Failure to parse tags from meshio.


In [3]:
mesh

<skfem MeshTet1 object>
  Number of elements: 289381
  Number of vertices: 48526
  Number of nodes: 48526
  Named subdomains [# elements]: air [217700], material [71681]

### Solving PDE in weak form

$$
\int_{\Omega} \nabla u \cdot \nabla v \ \mathrm{d}x = \int_{\Omega} \mathbf{M} \cdot \nabla v \ \mathrm{d}x
$$

If we consider same piece-wise-linear function space for both magnetic scalar potential $u$ and components of $\mathbf{M}$, we can re-write the PDE as:
$$
\sum_{j=1}^{n} \sum_{i=1}^{n} v_j u_i \int_{\Omega_{ji}} \nabla\phi_{j} \cdot \nabla\phi_{i} \ \mathrm{d}x = \sum_{j=1}^{n} \sum_{k=1}^{n} v_j m_k \int_{\Omega_{ik}} \hat{\phi_{k}} \cdot \nabla\phi_{j} \ \mathrm{d}x
$$
This can be further re-written as a matrix-vector product:
$$
\mathbf{v}_{j}^{\mathrm{T}} \cdot \mathbf{\mathrm{A}}_{ij} \cdot \mathbf{u}_{i} = \mathbf{v}_{j}^{\mathrm{T}} \cdot \mathbf{\mathrm{B}}_{jk} \cdot \mathbf{m}_{k}
$$

In [4]:
function_space_scalar = skfem.Basis(mesh, skfem.ElementTetP1())

In [5]:
function_space_vector = skfem.Basis(mesh, skfem.ElementVectorH1(skfem.ElementTetP1()))

In [6]:
@skfem.BilinearForm
def stiffness_matrix(u, v, _):
    return skfem.helpers.dot(skfem.helpers.grad(u), skfem.helpers.grad(v))

In [7]:
@skfem.BilinearForm
def mixed_form_matrix(u, v, _):
    return skfem.helpers.dot(u, skfem.helpers.grad(v))

In [8]:
%%timeit
skfem.asm(stiffness_matrix, function_space_scalar)

483 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
S = skfem.asm(stiffness_matrix, function_space_scalar)

In [10]:
%%timeit
skfem.asm(mixed_form_matrix, function_space_vector, function_space_scalar)

1.35 s ± 27.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
A = skfem.asm(mixed_form_matrix, function_space_vector, function_space_scalar)

In [12]:
S

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 733188 stored elements and shape (48526, 48526)>

In [13]:
A

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 2199189 stored elements and shape (48526, 145578)>

## Create initial magnetisation



In [14]:
import numpy as np

In [15]:
def mag_sat(x):
    m_x = np.zeros_like(x[0])
    m_y = np.zeros_like(x[1])
    m_z = np.zeros_like(x[2])
    m_z[(np.abs(x[0]) < 25.) & (np.abs(x[1]) < 25.) & (np.abs(x[2]) < 25.)] = 1.76 # T

    return np.stack(np.array([m_x, m_y, m_z]), axis=-1)

In [36]:
def mag_sat_2(x):
    m_x = np.zeros_like(x[0])
    m_y = np.zeros_like(x[1])
    m_z = np.zeros_like(x[2])
    m_z[(np.abs(x[0]) < 25.) & (np.abs(x[1]) < 25.) & (np.abs(x[2]) < 25.)] = 1.76 # T

    return np.array([m_x, m_y, m_z])

In [16]:
mag_init_arr = mag_sat(mesh.p)

In [21]:
mag_init = np.hstack(mag_init_arr)

In [35]:
mag_init

array([0.  , 0.  , 0.  , ..., 0.  , 0.  , 1.76], shape=(145578,))

In [37]:
mag_init_2 = function_space_vector.project(mag_sat_2)

In [47]:
np.split

array([ 0.00000000e+00,  0.00000000e+00, -7.82810710e-18, ...,
        0.00000000e+00,  0.00000000e+00,  2.23822429e+00], shape=(145578,))

In [22]:
%%timeit
A @ mag_init

4.14 ms ± 113 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [23]:
mesh.save("mag.vtk", point_data={"m": mag_init_arr})

In [41]:
mesh.save("mag_2.vtk", point_data={"m": mag_reshape})

## Solve the PDE

In [24]:
# ilu_operator = skfem.utils.build_pc_ilu(S_cnd)

In [25]:
from scipy.sparse.linalg import spsolve

In [26]:
rhs = A @ mag_init

In [27]:
S_cnd, rhs_cnd, _, sol_indices = skfem.condense(A=S, b=rhs, D=mesh.boundary_nodes())

In [28]:
sol = spsolve(A=S_cnd, b=rhs_cnd)

In [29]:
np.linalg.norm(S_cnd @ sol - rhs_cnd)

np.float64(2.1587861049079834e-11)

## Calculate the $\mathbf{H}_\mathrm{dmg}$ from the scalar potential

In [30]:
pot = function_space_scalar.zeros()

In [31]:
pot[sol_indices] = sol

In [32]:
h_dmg = function_space_vector.project(-skfem.helpers.grad(function_space_scalar.interpolate(pot)))

In [33]:
h_dmg_reshape = np.stack(np.split(h_dmg, 3), axis=-1)

In [34]:
mesh.save("h_dmg.vtk", point_data={"h_dmg": h_dmg_reshape})